In [1]:
# https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle?scriptVersionId=1669479
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [10]:
print train.shape
train.head()

(19579, 3)


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [5]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


In [6]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [7]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [9]:
y
unique, counts = np.unique(y, return_counts=True)

print np.asarray((unique, counts)).T

[[   0 7900]
 [   1 5635]
 [   2 6044]]


In [11]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [12]:
print (xtrain.shape)
print (xvalid.shape)

(17621,)
(1958,)


In [14]:
unique, counts = np.unique(ytrain, return_counts=True)

print np.asarray((unique, counts)).T

[[   0 7110]
 [   1 5071]
 [   2 5440]]


In [15]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [16]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.626 


In [19]:
ytrain[:10]

array([2, 0, 2, 1, 2, 2, 1, 2, 1, 0])

In [15]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [17]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.527 


In [19]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [03:12, 11396.35it/s]

Found 2196016 word vectors.


In [20]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [21]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]


100%|██████████| 1958/1958 [00:01<00:00, 1391.54it/s]


In [22]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [25]:
print xtrain_glove[0:10]

[[ 0.00178168 -0.00567005  0.03136761 ..., -0.01590794  0.00661943
  -0.00128377]
 [-0.00907115  0.06936043 -0.05057131 ..., -0.0010203  -0.02541611
  -0.00590883]
 [-0.0002239  -0.0083171  -0.02808638 ...,  0.00225637  0.01857211
  -0.0032152 ]
 ..., 
 [ 0.04512988  0.00600871 -0.01274933 ...,  0.04182176 -0.02443818
  -0.00976713]
 [ 0.015107    0.03778527 -0.06078252 ...,  0.00794491 -0.00704576
   0.01568724]
 [ 0.00851711  0.06246101  0.01313653 ..., -0.07798822 -0.08899614
  -0.04397095]]


In [23]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [44]:
print yvalid.shape
print yvalid_enc.shape

(1958,)
(1958, 3)


In [24]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [26]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [27]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=True, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 17621 samples, validate on 1958 samples
Epoch 1/5
17621/17621 [==============================] - 5s 276us/step - loss: 0.9163 - val_loss: 0.7314
Epoch 2/5
17621/17621 [==============================] - 5s 287us/step - loss: 0.7047 - val_loss: 0.6973
Epoch 3/5
17621/17621 [==============================] - 5s 306us/step - loss: 0.6469 - val_loss: 0.6695
Epoch 4/5
17621/17621 [==============================] - 5s 312us/step - loss: 0.5941 - val_loss: 0.6542
Epoch 5/5
17621/17621 [==============================] - 6s 328us/step - loss: 0.5602 - val_loss: 0.6705


In [28]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [29]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 25943/25943 [00:00<00:00, 113933.45it/s]


In [30]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

In [32]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 272s 15ms/step - loss: 1.0645 - val_loss: 0.9139
Epoch 2/100
17621/17621 [==============================] - 260s 15ms/step - loss: 0.9094 - val_loss: 0.8068
Epoch 3/100
17621/17621 [==============================] - 255s 14ms/step - loss: 0.8637 - val_loss: 0.7588
Epoch 4/100
17621/17621 [==============================] - 249s 14ms/step - loss: 0.8258 - val_loss: 0.7321
Epoch 5/100
17621/17621 [==============================] - 246s 14ms/step - loss: 0.8135 - val_loss: 0.7171
Epoch 6/100
17621/17621 [==============================] - 250s 14ms/step - loss: 0.7865 - val_loss: 0.7111
Epoch 7/100
17621/17621 [==============================] - 253s 14ms/step - loss: 0.7613 - val_loss: 0.6945
Epoch 8/100
17621/17621 [==============================] - 256s 15ms/step - loss: 0.7513 - val_loss: 0.6666
Epoch 9/100
17621/17621 [==============================] - 238s 14ms/step - loss: 0.724

In [33]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 430s 24ms/step - loss: 1.0673 - val_loss: 0.9036
Epoch 2/100
17621/17621 [==============================] - 666s 38ms/step - loss: 0.9058 - val_loss: 0.7786
Epoch 3/100
17621/17621 [==============================] - 383s 22ms/step - loss: 0.8581 - val_loss: 0.8040
Epoch 4/100
17621/17621 [==============================] - 375s 21ms/step - loss: 0.8378 - val_loss: 0.7378
Epoch 5/100
17621/17621 [==============================] - 357s 20ms/step - loss: 0.8154 - val_loss: 0.7486
Epoch 6/100
17621/17621 [==============================] - 336s 19ms/step - loss: 0.7865 - val_loss: 0.6951
Epoch 7/100
17621/17621 [==============================] - 332s 19ms/step - loss: 0.7613 - val_loss: 0.6758
Epoch 8/100
17621/17621 [==============================] - 334s 19ms/step - loss: 0.7412 - val_loss: 0.6569
Epoch 9/100
17621/17621 [==============================] - 336s 19ms/step - loss: 0.709

In [34]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - 297s 17ms/step - loss: 1.0818 - val_loss: 0.9525
Epoch 2/100
17621/17621 [==============================] - 300s 17ms/step - loss: 0.9698 - val_loss: 0.8590
Epoch 3/100
17621/17621 [==============================] - 302s 17ms/step - loss: 0.8977 - val_loss: 0.7936
Epoch 4/100
17621/17621 [==============================] - 301s 17ms/step - loss: 0.8456 - val_loss: 0.7918
Epoch 5/100
17621/17621 [==============================] - 301s 17ms/step - loss: 0.8197 - val_loss: 0.7385
Epoch 6/100
17621/17621 [==============================] - 301s 17ms/step - loss: 0.7724 - val_loss: 0.6974
Epoch 7/100
17621/17621 [==============================] - 302s 17ms/step - loss: 0.7619 - val_loss: 0.6873
Epoch 8/100
17621/17621 [==============================] - 303s 17ms/step - loss: 0.7265 - val_loss: 0.6445
Epoch 9/100
17621/17621 [==============================] - 343s 19ms/step - loss: 0.705

In [40]:
xtrain_pad

array([[   0,    0,    0, ...,   27,   29,  166],
       [   0,    0,    0, ...,   86, 1682,  561],
       [   0,    0,    0, ...,   27,   15,  385],
       ..., 
       [   0,    0,    0, ...,    2,    1, 2734],
       [   0,    0,    0, ...,  403,   12, 2136],
       [   0,    0,    0, ...,   90,   63, 2787]], dtype=int32)

In [38]:
len(word_index)

25943